In [1]:
library(MatchIt)
library(survival)
library(survminer)
library(tidyr)
library(tableone)
library(TriMatch)

Loading required package: ggplot2

Loading required package: ggpubr


Attaching package: ‘survminer’


The following object is masked from ‘package:survival’:

    myeloma


Loading required package: scales

Loading required package: reshape2


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: ez



In [2]:
df_psm <- read.csv("df_diuretic_responsitive.csv",as.is = TRUE)
df_psm <- as.data.frame(df_psm)
df_psm$group_creteria <- df_psm[ , "label_diu_res"]
df_psm$label <- df_psm[ , "label_diu_res"]
df_psm$group_creteria <- as.factor(df_psm$group_creteria)

# PSM

In [3]:
match_var1 <- c('creatinine', 'urineoutput','sofa_norenal',"colloid_bolus" )   #,'Congestive_heart_failure'
match_var2 <- c('creatinine', 'urineoutput','sofa_norenal',"colloid_bolus" )  
match_var3 <- c('creatinine', 'urineoutput','sofa_norenal',"colloid_bolus"  )  

# match_var1 <- c("colloid_bolus",'urineoutput','creatinine','sofa_norenal','weight','age','gcs','potassium','bicarbonate','temperature','hematocrit','heart_rate','resp_rate','lactate')  
# match_var2 <- c("colloid_bolus",'urineoutput','creatinine','sofa_norenal','weight','age','gcs','potassium','bicarbonate','temperature','hematocrit','heart_rate','resp_rate','lactate') 
# match_var3 <- c("colloid_bolus",'urineoutput','creatinine','sofa_norenal','weight','age','gcs','potassium','bicarbonate','temperature','hematocrit','heart_rate','resp_rate','lactate')

print(table(df_psm$group_creteria))
df <- df_psm


   No diuretic Non-responsive     responsive 
           640            392           1151 


In [4]:
df_ <- df[df$groupHPD==1,]
print(table(df_$group_creteria))
formu <- paste(match_var1, collapse = " + ")
formu <- paste("~", formu)

df_.tpsa <- trips(df_, df_$group_creteria, as.formula(formu))
df_.matched.1to1 <- trimatch(df_.tpsa,caliper=0.1)# ,method=OneToN, M1=, M2=2) # ,caliper=0.13   
# ,exact=df_[,c('creatinine','urineoutput','sofa_norenal','colloid_bolus','Congestive_heart_failure')]
# df_.matched.1to1 <- df_.matched.1to1[!duplicated(df_.matched.1to1[,1]),]
# df_.matched.1to1 <- df_.matched.1to1[!duplicated(df_.matched.1to1[,2]),]                                    
# df_.matched.1to1 <- df_.matched.1to1[!duplicated(df_.matched.1to1[,3]),]
df_merge <- cbind(df_,df_.tpsa$id) 
matched_id <- c(df_.matched.1to1[, 1],df_.matched.1to1[,2],df_.matched.1to1[,3])
df_match <- df_merge[df_merge$`df_.tpsa$id` %in% matched_id, ]
print(table(df_match$group_creteria))
table1 <- CreateTableOne(vars = match_var1, data = df_match, strata = "group_creteria")  
tab1_csv <- print(table1)
df_match1 <- df_match


   No diuretic Non-responsive     responsive 
           119            138            307 


Using logistic regression to estimate propensity scores...




   No diuretic Non-responsive     responsive 
            97            107            102 
                           Stratified by group_creteria
                            No diuretic       Non-responsive   
  n                              97               107          
  creatinine (mean (SD))       1.35 (0.49)       1.49 (0.50)   
  urineoutput (mean (SD))   1253.90 (965.60)  1223.93 (1119.83)
  sofa_norenal (mean (SD))     6.98 (4.15)       7.54 (3.40)   
  colloid_bolus (mean (SD)) 4089.41 (2482.58) 4120.02 (2639.98)
                           Stratified by group_creteria
                            responsive        p      test
  n                             102                      
  creatinine (mean (SD))       1.49 (0.45)     0.062     
  urineoutput (mean (SD))   1250.52 (842.51)   0.972     
  sofa_norenal (mean (SD))     6.97 (2.95)     0.410     
  colloid_bolus (mean (SD)) 3953.69 (2219.84)  0.875     


In [5]:
df_ <- df[df$groupHPD==2,]
print(table(df_$group_creteria))
formu <- paste(match_var2, collapse = " + ")
formu <- paste("~", formu)

df_.tpsa <- trips(df_, df_$group_creteria, as.formula(formu))
#df_.matched.1to1 <- trimatch(df_.tpsa, method=OneToN, M1=3, M2=3)#  
df_.matched.1to1 <- trimatch(df_.tpsa, caliper=0.03)#  method=OneToN, M1=2, M2=2
# df_.matched.1to1 <- df_.matched.1to1[!duplicated(df_.matched.1to1[,1]),]
# df_.matched.1to1 <- df_.matched.1to1[!duplicated(df_.matched.1to1[,2]),]
# df_.matched.1to1 <- df_.matched.1to1[!duplicated(df_.matched.1to1[,3]),]
df_merge <- cbind(df_,df_.tpsa$id)
matched_id <- c(df_.matched.1to1[, 1],df_.matched.1to1[,2],df_.matched.1to1[,3])
df_match <- df_merge[df_merge$`df_.tpsa$id` %in% matched_id, ]
print(table(df_match$group_creteria))
table2 <- CreateTableOne(vars = match_var2, data = df_match, strata = "group_creteria")  
tab2_csv <- print(table2)
df_match2 <- df_match


   No diuretic Non-responsive     responsive 
           445            176            779 


Using logistic regression to estimate propensity scores...




   No diuretic Non-responsive     responsive 
           152             91            124 
                           Stratified by group_creteria
                            No diuretic       Non-responsive   
  n                             152                91          
  creatinine (mean (SD))       0.94 (0.24)       0.91 (0.25)   
  urineoutput (mean (SD))   1504.13 (726.71)  1520.51 (785.75) 
  sofa_norenal (mean (SD))     4.76 (2.35)       5.15 (2.59)   
  colloid_bolus (mean (SD)) 3049.06 (1563.83) 2850.08 (1236.83)
                           Stratified by group_creteria
                            responsive        p      test
  n                             124                      
  creatinine (mean (SD))       0.93 (0.22)     0.650     
  urineoutput (mean (SD))   1459.93 (660.41)   0.807     
  sofa_norenal (mean (SD))     5.13 (2.46)     0.342     
  colloid_bolus (mean (SD)) 2947.41 (1030.50)  0.517     


In [6]:
df_ <- df[df$groupHPD==3,]
print(table(df_$group_creteria))
formu <- paste(match_var3, collapse = " + ")
formu <- paste("~", formu)

df_.tpsa <- trips(df_, df_$group_creteria, as.formula(formu))
df_.matched.1to1 <- trimatch(df_.tpsa, caliper=0.2 )#
# df_.matched.1to1 <- df_.matched.1to1[!duplicated(df_.matched.1to1[,1]),]
# df_.matched.1to1 <- df_.matched.1to1[!duplicated(df_.matched.1to1[,2]),]
# df_.matched.1to1 <- df_.matched.1to1[!duplicated(df_.matched.1to1[,3]),]
df_merge <- cbind(df_,df_.tpsa$id)
matched_id <- c(df_.matched.1to1[, 1],df_.matched.1to1[,2],df_.matched.1to1[,3])
df_match <- df_merge[df_merge$`df_.tpsa$id` %in% matched_id, ]
print(table(df_match$group_creteria))
table3 <- CreateTableOne(vars = match_var3, data = df_match, strata = "group_creteria")  
tab3_csv <- print(table3)
df_match3 <- df_match


   No diuretic Non-responsive     responsive 
            76             78             65 


Using logistic regression to estimate propensity scores...




   No diuretic Non-responsive     responsive 
            64             41             48 
                           Stratified by group_creteria
                            No diuretic       Non-responsive   
  n                              64                41          
  creatinine (mean (SD))       1.58 (0.59)       1.52 (0.54)   
  urineoutput (mean (SD))    807.30 (704.19)   973.15 (706.53) 
  sofa_norenal (mean (SD))     9.61 (3.11)       9.48 (3.29)   
  colloid_bolus (mean (SD)) 4787.79 (2418.10) 4555.75 (2015.20)
                           Stratified by group_creteria
                            responsive        p      test
  n                              48                      
  creatinine (mean (SD))       1.60 (0.56)     0.820     
  urineoutput (mean (SD))    975.20 (626.83)   0.330     
  sofa_norenal (mean (SD))     9.02 (2.71)     0.586     
  colloid_bolus (mean (SD)) 4649.09 (2644.27)  0.883     


In [7]:
df_match <- rbind(df_match1,df_match2,df_match3)
# matched_id <- c(df_match$stay_id)
# df_match_add <- df_psm[df_psm$stay_id %in% matched_id, ]
write.csv(df_match, file = "df_diuretic_responsitive_match.csv", row.names = FALSE)

In [8]:
tab1_csv1 <- cbind(tab1_csv, rep("C1", nrow(tab1_csv)))
tab2_csv1 <- cbind(tab2_csv, rep("C2", nrow(tab2_csv)))
tab3_csv1 <- cbind(tab3_csv, rep("C3", nrow(tab3_csv)))
tab_combind <- rbind(tab1_csv1,tab2_csv1,tab3_csv1)
colnames(tab_combind) <- c("features",'No diuretic','Non-responsive','responsive',"p-value","subphenotypes")
write.csv(tab_combind, file = "aumcdb_3psm_macth.csv")
tab_combind

,features,No diuretic,Non-responsive,responsive,p-value,subphenotypes
n,97,107,102,,,C1
creatinine (mean (SD)),1.35 (0.49),1.49 (0.50),1.49 (0.45),0.062,,C1
urineoutput (mean (SD)),1253.90 (965.60),1223.93 (1119.83),1250.52 (842.51),0.972,,C1
sofa_norenal (mean (SD)),6.98 (4.15),7.54 (3.40),6.97 (2.95),0.410,,C1
colloid_bolus (mean (SD)),4089.41 (2482.58),4120.02 (2639.98),3953.69 (2219.84),0.875,,C1
n,152,91,124,,,C2
creatinine (mean (SD)),0.94 (0.24),0.91 (0.25),0.93 (0.22),0.650,,C2
urineoutput (mean (SD)),1504.13 (726.71),1520.51 (785.75),1459.93 (660.41),0.807,,C2
sofa_norenal (mean (SD)),4.76 (2.35),5.15 (2.59),5.13 (2.46),0.342,,C2
colloid_bolus (mean (SD)),3049.06 (1563.83),2850.08 (1236.83),2947.41 (1030.50),0.517,,C2


# survival

In [9]:
# df_match = read.csv("df_diuretic_responsitive.csv",as.is = TRUE)
# df_match <- as.data.frame(df_match)
# df_match$group_creteria <- df_match[ , "label_diu_res"]
# df_match$group_creteria <- as.factor(df_match$group_creteria)

In [10]:
df_survival =  df_match[,c("survival_28day","mortality_28d", "groupHPD","group_creteria")]

In [11]:
survival_curve <- function(data,title,survival_day,expire_flag){
    df_select <- data
    fit <- survfit(Surv(survival_day,expire_flag) ~ label, data = df_select)
    ggsurvplot(fit, data = df_select,
               surv.median.line = hv,  # 增加中位生存时间,
               conf.int = TRUE, # 增加置信区间,
              # risk.table = TRUE, # 绘制累计风险曲线,
#               title=title,,
               pval = TRUE,
               #add.all = TRUE, # 添加总患者生存曲线,
               xlab = "Follow up time (days)",
               legend = c(0.2,0.2)) # 指定图例位置,
}

# df_select <- df_survival
# #df_select = df_select[df_select$group_creteria!="No diuretic",]
# fit <- survfit(Surv(survival_28day,mortality_28d) ~ group_creteria, data = df_select)
# ggsurvplot(fit, data = df_select, 
#                surv.median.line = "hv",  # 增加中位生存时间,
#                conf.int = TRUE, # 增加置信区间,
#                risk.table = TRUE, # 绘制累计风险曲线,
#                title="ALL: 28 day-Mortality - MIMIC",
#                pval = TRUE,
#                #add.all = TRUE, # 添加总患者生存曲线,
#                xlab = "Follow up time(day)",
#                legend = c(0.8,0.2)) # 指定图例位置

In [17]:
pdf("AUMC_C1_28d_mortality.pdf")
png("AUMC_C1_28d_mortality.png")

df_select <- df_survival[df_survival$groupHPD==1,]
#df_select = df_select[df_select$group_creteria!="No diuretic",]

fit <- survfit(Surv(survival_28day,mortality_28d) ~ group_creteria, data = df_select)
ggsurv <- ggsurvplot(fit, data = df_select, 
#                surv.median.line = "hv",  # 增加中位生存时间,
               conf.int = TRUE, # 增加置信区间,
#               risk.table = TRUE, # 绘制累计风险曲线,
               title="AUMC: DR phenotype",
               palette = c("#f8746a" ,"#74a8ff","#3fc968"),
               pval = TRUE,pval.size = 10,
               #add.all = TRUE, # 添加总患者生存曲线,
               xlab = "Follow up time (days)",
               legend = "none",#legend = c(0.8,0.2),legend.title = "",legend.labs = c("No diuretic", "Non-Responsive","Responsive"),
                font.main = 25,     #标题字体大小
               font.x = 27,        #x坐标字体大小
               font.y = 27,        #y坐标字体大小
               font.tickslab = 16,
               risk.table.fontsize = 16,
tables.theme = theme_cleantable()+ theme( ))

ggsurv$plot <- ggsurv$plot +
  theme(legend.text = element_text(size = 16),axis.text.y = element_text(size = 25), axis.text.x = element_text(size = 25))  # 调整图例文本的字体大小为16
ggsurv$table <- ggsurv$table +
  theme(plot.title = element_text(size = 20, color = "black", face = "bold"))
ggsurv
dev.off()

png 
  2

In [13]:
pdf("AUMC_C2_28d_mortality.pdf")
png("AUMC_C2_28d_mortality.png")
df_select <- df_survival[df_survival$groupHPD==2,]
#df_select = df_select[df_select$group_creteria!="No diuretic",] #Non-responsive
 
fit <- survfit(Surv(survival_28day,mortality_28d) ~ group_creteria, data = df_select)
ggsurv <- ggsurvplot(fit, data = df_select, 
#                surv.median.line = "hv",  # 增加中位生存时间,
               conf.int = TRUE, # 增加置信区间,
#               risk.table = TRUE, # 绘制累计风险曲线,
               title="AUMC: RR phenotype",
               pval = TRUE,pval.size = 10,
               #add.all = TRUE, # 添加总患者生存曲线,
               xlab = "Follow up time (days)",
               palette = c("#f8746a" ,"#74a8ff","#3fc968"),
               legend = "none",#legend = c(0.8,0.2),legend.title = "",legend.labs = c("No diuretic", "Non-Responsive","Responsive"),
                font.main = 25,     #标题字体大小
               font.x = 27,        #x坐标字体大小
               font.y = 27,        #y坐标字体大小
               font.tickslab = 16,
               risk.table.fontsize = 16,
tables.theme = theme_cleantable()+ theme( ))

ggsurv$plot <- ggsurv$plot +
  theme(legend.text = element_text(size = 16),axis.text.y = element_text(size = 25), axis.text.x = element_text(size = 25))  # 调整图例文本的字体大小为16
ggsurv$table <- ggsurv$table +
  theme(plot.title = element_text(size = 20, color = "black", face = "bold"))
ggsurv
dev.off()

png 
  2

In [14]:
pdf("AUMC_C3_28d_mortality.pdf")
png("AUMC_C3_28d_mortality.png")

df_select <- df_survival[df_survival$groupHPD==3,]
#df_select = df_select[df_select$group_creteria!="No diuretic",]

fit <- survfit(Surv(survival_28day,mortality_28d) ~ group_creteria, data = df_select)
ggsurv <- ggsurvplot(fit, data = df_select, 
#                surv.median.line = "hv",  # 增加中位生存时间,
               conf.int = TRUE, # 增加置信区间,
          #     risk.table = TRUE, # 绘制累计风险曲线,
               title="AUMC: PW phenotype",
               pval = TRUE,pval.size = 10,
               #add.all = TRUE, # 添加总患者生存曲线,
               xlab = "Follow up time (days)",
               palette = c("#f8746a" ,"#74a8ff","#3fc968"), # 红 蓝 绿
               legend = "none",
               #legend = c(0.8,0.2),legend.title = "",legend.labs = c("No diuretic", "Non-Responsive","Responsive"),
                font.main = 25,     #标题字体大小
               font.x = 27,        #x坐标字体大小
               font.y = 27,        #y坐标字体大小
               font.tickslab = 16,
               risk.table.fontsize = 16,
tables.theme = theme_cleantable()+ theme( ))

ggsurv$plot <- ggsurv$plot +
  theme(legend.text = element_text(size = 16),axis.text.y = element_text(size = 25), axis.text.x = element_text(size = 25))  # 调整图例文本的字体大小为16
ggsurv$table <- ggsurv$table +
  theme(plot.title = element_text(size = 20, color = "black", face = "bold"))
ggsurv
dev.off()

png 
  2

In [15]:
# 产生legend
df_select <- df_survival[df_survival$groupHPD==1,]
#df_select = df_select[df_select$group_creteria!="No diuretic",]
pdf("AUMC_legend_28d_mortality.pdf")
fit <- survfit(Surv(survival_28day,mortality_28d) ~ group_creteria, data = df_select)
ggsurv <- ggsurvplot(fit, data = df_select, 
               surv.median.line = "hv",  # 增加中位生存时间,
               conf.int = TRUE, # 增加置信区间,
#               risk.table = TRUE, # 绘制累计风险曲线,
               title="AUMC: DR phenotype",
               palette = c("#f8746a" ,"#74a8ff","#3fc968"),
               pval = TRUE,
               #add.all = TRUE, # 添加总患者生存曲线,
               xlab = "Follow up time (days)",
               legend.title = "",legend.labs = c("No diuretic group", "Non-Responsive","Responsive"),#) # 指定图例位置 
                           font.main = 25,     #标题字体大小
               font.x = 20,        #x坐标字体大小
               font.y = 20,        #y坐标字体大小
               font.tickslab = 16,
               risk.table.fontsize = 6,tables.theme = theme_cleantable()+ 
    theme(axis.text.y = element_text(size = 16))) # 指定图例位置
ggsurv$plot <- ggsurv$plot +
  theme(legend.text = element_text(size = 16))  # 调整图例文本的字体大小为16
ggsurv$table <- ggsurv$table +
  theme(plot.title = element_text(size = 20, color = "black", face = "bold"))
ggsurv
dev.off()

png 
  2